# Spark SQL Quiz

This quiz uses the same dataset and questions from the Spark Data Frames Programming Quiz. For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions

spark = SparkSession \
    .builder \
    .appName("Spark SQL Quiz") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [2]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [3]:
# create temporary view to run sql queries
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1
spark.sql('SELECT DISTINCT page FROM user_log_table WHERE userId = ""').show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [7]:
# TODO: write your code to answer question 3
spark.sql('SELECT COUNT(DISTINCT userId) FROM user_log_table WHERE gender="F"').show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [12]:
# TODO: write your code to answer question 4
spark.sql('SELECT artist, COUNT(artist) AS numPlays \
FROM user_log_table \
GROUP BY artist \
ORDER BY numPlays DESC \
LIMIT 1').show()

+--------+--------+
|  artist|numPlays|
+--------+--------+
|Coldplay|      83|
+--------+--------+



In [17]:
# alternative solution
# get artist play count
play_counts = spark.sql('SELECT artist, COUNT(artist) AS numPlays \
FROM user_log_table \
GROUP BY artist')

# save the results in a new view
play_counts.createOrReplaceTempView("artist_counts")

# use a self join to find where the max play equals the count value
spark.sql('SELECT a2.artist, a2.numPlays \
FROM (SELECT MAX(numPlays) AS max_plays FROM artist_counts) as a1 \
JOIN artist_counts AS a2 \
ON a1.max_plays = a2.numPlays').show()

+--------+--------+
|  artist|numPlays|
+--------+--------+
|Coldplay|      83|
+--------+--------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [30]:
# TODO: write your code to answer question 5

is_home = spark.sql('SELECT userId, page, ts, CASE WHEN page = "Home" THEN 1 ELSE 0 END AS is_home \
FROM user_log_table \
WHERE (page="NextSong") or (page = "Home")')

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql('SELECT *, SUM(is_home) OVER \
(PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
FROM is_home_table')

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

In [27]:
cumulative_sum.show()

+------+--------+-------------+-------+------+
|userId|    page|           ts|is_home|period|
+------+--------+-------------+-------+------+
|  1436|NextSong|1513783259284|      0|     0|
|  1436|NextSong|1513782858284|      0|     0|
|  2088|    Home|1513805972284|      1|     1|
|  2088|NextSong|1513805859284|      0|     1|
|  2088|NextSong|1513805494284|      0|     1|
|  2088|NextSong|1513805065284|      0|     1|
|  2088|NextSong|1513804786284|      0|     1|
|  2088|NextSong|1513804555284|      0|     1|
|  2088|NextSong|1513804196284|      0|     1|
|  2088|NextSong|1513803967284|      0|     1|
|  2088|NextSong|1513803820284|      0|     1|
|  2088|NextSong|1513803651284|      0|     1|
|  2088|NextSong|1513803413284|      0|     1|
|  2088|NextSong|1513803254284|      0|     1|
|  2088|NextSong|1513803057284|      0|     1|
|  2088|NextSong|1513802824284|      0|     1|
|  2162|NextSong|1513781246284|      0|     0|
|  2162|NextSong|1513781065284|      0|     0|
|  2162|NextS

In [31]:
# find the average count for NextSong
spark.sql('SELECT AVG(count_results) \
FROM (SELECT COUNT(*) AS count_results \
FROM period_table \
GROUP BY userId, period, page HAVING page = "NextSong") AS counts').show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

